In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import time
from helpers import permute_data, random_like

In [2]:
from helpers import download_data
from pathlib import Path

In [3]:
from itertools import product, combinations, permutations 

In [4]:
tickers = [
    "NTPC.NS","HINDALCO.NS","BHARTIARTL.NS",
    "SHREECEM.NS","TCS.NS","HDFCLIFE.NS",
    "CIPLA.NS","LT.NS","ULTRACEMCO.NS",
    "WIPRO.NS","KOTAKBANK.NS","BAJFINANCE.NS",
    "TATACONSUM.NS","BAJAJFINSV.NS","MARUTI.NS",
    "TITAN.NS","ICICIBANK.NS","ONGC.NS","ITC.NS",
    "APOLLOHOSP.NS","BRITANNIA.NS","BAJAJ-AUTO.NS",
    "TECHM.NS","COALINDIA.NS","TATASTEEL.NS","HEROMOTOCO.NS",
    "INDUSINDBK.NS","NESTLEIND.NS","M&M.NS","RELIANCE.NS"
]

In [5]:
START_DATE = pd.to_datetime("2009-10-01")
END_DATE = pd.to_datetime("2020-01-31")

In [8]:
DATA_PATH = Path("./data/").resolve()
titan_price = pd.read_parquet(f"{DATA_PATH}/TITAN.NS.parquet").get("Close").loc[START_DATE:END_DATE]

## PARTITION OURSELVES

In [9]:
START_DATE_1  = pd.to_datetime("2010-01-01")
END_DATE_1   = pd.to_datetime("2013-01-01")

START_DATE_2 = pd.to_datetime("2013-01-01")
END_DATE_2 = pd.to_datetime("2016-01-01")

START_DATE_3 = pd.to_datetime("2016-01-01")
END_DATE_3 = pd.to_datetime("2019-01-01")

PARTITION_1 = titan_price.loc[START_DATE_1 : END_DATE_1]
PARTITION_2 = titan_price.loc[START_DATE_2 : END_DATE_2]
PARTITION_3 = titan_price.loc[START_DATE_3 : END_DATE_3]

## PARTITION WITH VBT

In [10]:
BBANDS = vbt.IndicatorFactory.from_pandas_ta("BBANDS")
lengths, stds, mas = vbt.utils.params.create_param_combs( (product, np.arange(10, 55, 5),  (product, [2, 3],["sma", "ema"]) ) ) 
bbands = BBANDS.run(PARTITION_1, length=lengths, std=stds, mamode = mas)

In [11]:
entries = bbands.close_below(bbands.bbl)
exits = bbands.close_above(bbands.bbu)

In [82]:
pf = vbt.Portfolio.from_signals(PARTITION_1, entries, exits, fees=0.001, freq='1D')

In [ ]:
pf = vbt.Portfolio.from_signals(PARTITION_1, entries, exits, fees=0.001, freq='1D')

In [103]:
pf.trades.expectancy()

bbands_length  bbands_std  bbands_mamode
10             2           sma               6.026686
                           ema              -2.958704
               3           sma                    NaN
                           ema                    NaN
15             2           sma               5.701204
                           ema               7.784681
               3           sma              18.725320
                           ema                    NaN
20             2           sma               6.104471
                           ema               7.097061
               3           sma              24.275163
                           ema                    NaN
25             2           sma               6.167754
                           ema               6.132239
               3           sma              27.849666
                           ema                    NaN
30             2           sma               8.561957
                           ema           

In [84]:
pf.trades.expectancy().mean()

25.637995937037743

In [93]:
from strategies.strategy import Strategy

class BBANDStrategy(Strategy):
    def __init__(self, data):
        self.data = data
        
    def init(self):
        lengths, stds, mas = vbt.utils.params.create_param_combs((product, np.arange(10, 55, 5),  (product, [2, 3],["sma", "ema"]) ) ) 
        self.indicator = vbt.IndicatorFactory.from_pandas_ta("BBANDS").run(
            self.data,
            length=lengths,
            std=stds,
            mamode = mas
        )
        
    
    def get_entries(self):
        self.init()
        return self.indicator.close_below(bbands.bbl)
        
       
    
    def get_exits(self):
        return self.indicator.close_above(bbands.bbu)
          

In [15]:
class Backtest:
    def __init__(self, ticker:str, strategy:Strategy, fees:float = 0.001, size:float = np.inf, freq:str = "1D"):
        self.ticker=ticker
        self.strategy = strategy
        self.fees = fees
        self.size = size
        self.freq=freq
    
    def __repr__(self):
        return f"<Bactest {str(self.strategy)}>"
    
    def _get_portfolio(self, entries, exits):
        return vbt.Portfolio.from_signals(close=self.strategy.data,
                                          entries=entries, 
                                          exits=exits, 
                                          fees=self.fees,size = self.size, 
                                          freq=self.freq)
    
    def run(self, **kwargs):
        entries = self.strategy.get_entries()
        exits = self.strategy.get_exits()
        
        random_entries = entries.apply(random_like)
        permuted_entries = entries.apply(permute_data)
        
        portfolio = self._get_portfolio(entries, exits, **kwargs)
        random_porfolio = self._get_portfolio(random_entries, exits, **kwargs)
        permuted_portfolio = self._get_portfolio(permuted_entries, exits,**kwargs)
        
        output = {
            self.ticker : dict(
                mean_expectancy = portfolio.trades.expectancy().mean(),
                max_expectancy = portfolio.trades.expectancy().max(),
                mean_random_expectancy= random_porfolio.trades.expectancy().mean(),
                mean_permuted_expectancy = permuted_portfolio.trades.expectancy().mean()
            )
        }
        return output      

In [94]:
bbands_strategy = BBANDStrategy(PARTITION_1)

In [95]:
backtest = Backtest("TITAN", bbands_strategy)

In [96]:
output = backtest.run()

In [31]:
fast_windows, slow_windows, signal_windows = vbt.utils.params.create_param_combs(
    (product, (combinations, np.arange(2, 51, 1), 2), np.arange(2, 21, 1)))

# Run MACD indicator
macd_ind = vbt.MACD.run(
    close= PARTITION_1,
    fast_window=fast_windows,
    slow_window=slow_windows,
    signal_window=signal_windows
)

# Long when MACD is above zero AND signal
entries = macd_ind.macd_above(0) & macd_ind.macd_above(macd_ind.signal)

# Short when MACD is below zero OR signal
exits = macd_ind.macd_below(0) | macd_ind.macd_below(macd_ind.signal)

# Build portfolio
pf = vbt.Portfolio.from_signals(
    PARTITION_1.vbt.tile(len(fast_windows)), entries, exits, fees=0.001, freq='1D')

In [32]:
pf.trades.expectancy().mean()

2.015990013207552